In [0]:
%python
# Celda 1: Leer datos desde las APIs
import requests

url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv"
url_men = "https://www.datos.gov.co/resource/nudc-7mev.csv"

# Descargar los archivos CSV
response_secop = requests.get(url_secop)
response_men = requests.get(url_men)

# Guardar los archivos CSV en DBFS
dbutils.fs.put("/dbfs/tmp/secop.csv", response_secop.content.decode('utf-8'), overwrite=True)
dbutils.fs.put("/dbfs/tmp/men.csv", response_men.content.decode('utf-8'), overwrite=True)

# Usamos spark.read para ingerir los datos.
# option("header", "true") indica que la primera fila contiene los nombres de las columnas.
# option("inferSchema", "true") le pide a Spark que adivine el tipo de cada columna.
df_secop = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/dbfs/tmp/secop.csv")
df_men = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/dbfs/tmp/men.csv")

# display() es un comando especial de Databricks para una visualización interactiva.
print("Datos del SECOP cargados:")
display(df_secop)

print("Datos del MEN cargados:")
display(df_men)

In [0]:
%fs ls dbfs:/Volumes/workspace/default/
 

In [0]:
# Paso 1: Descargar los datos con requests y leerlos en pandas
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=10000"
url_men = "https://www.datos.gov.co/resource/nudc-7mev.csv?$limit=10000"
 

# Descargar contenido
response_secop = requests.get(url_secop)
response_men = requests.get(url_men)

# Convertir contenido a pandas usando StringIO
df_secop_pd = pd.read_csv(StringIO(response_secop.text))
df_men_pd = pd.read_csv(StringIO(response_men.text))

# Convertir pandas a Spark
df_secop = spark.createDataFrame(df_secop_pd)
df_men = spark.createDataFrame(df_men_pd)

# Mostrar en Databricks
display(df_secop)
display(df_men)
 

In [0]:
# Celda 2: Guardar los DataFrames como tablas Delta
# La función .saveAsTable() guarda los datos y registra la tabla en el Unity Catalog.
# El modo "overwrite" reemplaza la tabla si ya existe, ideal para actualizaciones.
df_secop.write.format("delta").mode("overwrite").saveAsTable("main.diplomado.secop")
df_men.write.format("delta").mode("overwrite").saveAsTable("main.diplomado.men_estadisticas")

print("¡Tablas guardadas exitosamente en el catálogo 'main', esquema 'diplomado_datos'!")